In [2]:
#import libraries

import requests
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
!pip install beautifulsoup4

distributed 1.21.8 requires msgpack, which is not installed.


In [4]:
!pip install lxml

distributed 1.21.8 requires msgpack, which is not installed.


In [5]:
!pip install html5lib

distributed 1.21.8 requires msgpack, which is not installed.


In [6]:
!pip install requests

distributed 1.21.8 requires msgpack, which is not installed.


In [7]:
from bs4 import BeautifulSoup

In [8]:
#Assign website link to a variable
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [9]:
#Use the function get from the requests library to download the Wikipedia page using the wikipedia_link as an argument. Assign the object to the variable postal_code_page.

#hint: requests.get()
source = requests.get(wikipedia_link).text
soup= BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [10]:
table = soup.table
#print(table)
postheader = table.find_all('tr')
#print(postheader)
 

In [11]:
! pip install tabulate

distributed 1.21.8 requires msgpack, which is not installed.


Get the data into a dataframe "DATA"

In [12]:
from tabulate import tabulate
table = soup.find_all('table')[0] 
df_list = pd.read_html(str(table))
#df_list[0]
type(df_list)
#print( tabulate(df_list[0], headers='keys', tablefmt='psql') )

data = pd.DataFrame(df_list[0])
type(data)
#data # 290*3

new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

type(data) #289*3
#data

pandas.core.frame.DataFrame

Clean up data where data is NOT ASSIGNED

In [13]:
data = data[data.Borough != 'Not assigned']
data.head() #212*3

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Massage Neighbouhood data to not to have NOT ASSIGNED value

In [14]:
#data['Neighbourhood'].loc[data['Neighbourhood'] == 'Not assigned'] = data[data.Borough]
#data.loc[data['Neighbourhood'] == 'Not assigned','Neighbourhood'] = data.loc[data[Borough]]
data['Neighbourhood'] = data.apply(
    lambda row: row['Borough'] if (row['Neighbourhood']=='Not assigned') else row['Neighbourhood'],
    axis=1
)
data.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Grouping the records with the same postal code and have concatenated Neighbourhood values

In [15]:
data_group = data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
data_group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood]],Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
data_group.shape

(103, 3)

Get Latitude and Longitude values for each neighbourhood

In [17]:

for index, row in data_group.iterrows():
    address_1 = row['Neighbourhood'] 
    address_2 = address_1.split(',')[-1]
    address_3 = address_2+","+"Ontario,Canada"
    #print(address_3) #-- It worked

In [18]:
column_names = ['Latitude', 'Longitude'] 
n_hood = pd.DataFrame(columns=column_names)
n_hood.shape

(0, 2)

Create a data frame with Latitude and Longitude values

In [19]:


for index, row in data_group.iterrows():
    try:
        address_1 = row['Neighbourhood'] 
        address_2 = address_1.split(',')[-1]
        address = address_2+","+"Ontario,Canada"
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(row['Borough'],address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        n_hood
        pass
    except ValueError as error_message:
        print("Error")
    except AttributeError:
        #print("Problem with data or cannot Geocode.")
        address_3 = row['Borough']
        address = address_3+","+"Ontario,Canada"
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
       # print(row['Borough'],address, latitude, longitude)
        n_hood
        pass



/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [26]:
n_hood.head()


,Latitude,Longitude
0,43.809196,-79.221701
1,43.775504,-79.134976
2,43.768914,-79.187291
3,43.759824,-79.225291
4,43.756467,-79.226692


In [ ]:
Concatenate the latitude and longitude values to the main dataframe to complete the results

In [27]:
df = pd.concat([data_group, n_hood[['Latitude', 'Longitude']]], axis=1)
df.shape
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.809196,-79.221701
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.775504,-79.134976
2,M1E,Scarborough,"Guildwood]],Morningside,West Hill",43.768914,-79.187291
3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692
5,M1J,Scarborough,Scarborough Village,43.743742,-79.211632
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.724878,-79.253969
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.697174,-79.274823
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.743742,-79.211632
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.711170,-79.248177


In [28]:
df.shape

(103, 5)